In [7]:
# Load packages and settings
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 15
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
from matplotlib import cm # Colormaps
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

import os
# import csv
import math

from datetime import date


saveFigures = True
# saveFigures = False
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

# Define running mean functions
def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
    
    
# Define paths
rootdir_data = os.getcwd() +"\\..\\DanskeData\\" 

path_data = rootdir_data + "ssi_data\\"
path_dash = rootdir_data + "ssi_dashboard\\"
path_vacc = rootdir_data + "ssi_vacc\\"
path_owid = os.getcwd() +"\\..\\InternationalData\\data\\" 

path_figs = os.getcwd() +"\\..\\Figures\\" 

saveFigures is set to: True
Done loading packages


In [8]:
df = pd.read_csv(path_owid+'OWID_data.csv')

In [9]:
# df.columns
df.location.unique()

curCountries = ['Denmark','European Union','Estonia','Latvia','Lithuania','Germany','France','Italy','Spain','Portugal','United Kingdom','United States']
# curCountries = ['Denmark','Estonia','Latvia','Lithuania','Germany','France','Italy','Spain','Portugal','United Kingdom','United States']

curCountries = np.array(curCountries)

allCountries = df[df.continent == 'Europe'].location.unique()

In [10]:
# plt.figure()
# curdf
# curVacc.isna() 


In [11]:
# curCountry = curCountries[0]

vaccRates = []
infRates = []
# for curCountry in curCountries:
curCountries = []
for curCountry in allCountries:
    curdf = df[df.location == curCountry]

    curInf = curdf.total_cases_per_million
    curVacc = curdf.people_vaccinated_per_hundred
    if len(curInf[~curInf.isna()]) == 0:
        continue
    if len(curVacc[~curVacc.isna()]) == 0:
        continue
    
    latestInfRate = curInf[~curInf.isna()].iloc[-1]
    infRates.append(100*latestInfRate/1000000) # Per million, changed to per hundred
    
    # curLastRow = curdf[curdf.date == (curdf.date.max())]
    # curdf.people_vaccinated_per_hundred
    latestVaccRate = curVacc[~curVacc.isna()].iloc[-1]
    vaccRates.append(latestVaccRate)
    
    # If loop didn't break yet, add to list of countries to show
    curCountries.append(curCountry)
    
    

vaccRates = np.array(vaccRates)    
infRates = np.array(infRates)
curCountries = np.array(curCountries)

# # Sort by vaccinationrate
# newOrder = np.argsort(vaccRates)
# vaccRates = vaccRates[newOrder]
# infRates = infRates[newOrder] 
# curCountries = curCountries[newOrder]

immuRateDependent = vaccRates + infRates
immuRateIndependent = vaccRates + infRates - (vaccRates/100)*(infRates/100)*100
infRateIndependent = infRates - (vaccRates/100)*(infRates/100)*100

In [12]:


fig,ax1 = plt.subplots(figsize=(20,8))

fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')

cmap = plt.cm.get_cmap('magma',5)
ax1.bar(curCountries,vaccRates,color=cmap(0),label='Vaccinations')
ax1.bar(curCountries,infRates,bottom=vaccRates,color=cmap(2),label='Cases (Dependent)')
ax1.bar(curCountries,infRateIndependent,bottom=vaccRates,color=cmap(1),label='Cases (Independent)')
# ax1.bar(curCountries,infRates,bottom=vaccRates,color=cmap(1),label='Cases (Dependent)')
# ax1.bar(curCountries,infRates - (vaccRates/100)*(infRates/100)*100,bottom=vaccRates,color=cmap(2),label='Cases (Independent)')
# ax1.bar(curCountries,vaccRates,color=cmap(0),label='Vaccinationer')
# ax1.bar(curCountries,infRates,bottom=vaccRates,color=cmap(1),label='Smittetilfælde (afhængig)')
# ax1.bar(curCountries,infRates - (vaccRates/100)*(infRates/100)*100,bottom=vaccRates,color=cmap(2),label='Smittetilfælde (uafhængig)')
# ax1.bar(curCountries,vaccRates,color='b',label='Vaccinationer')
# ax1.bar(curCountries,infRates,bottom=vaccRates,color='r',label='Smittetilfælde (afhængig)')
# ax1.bar(curCountries,infRates - (vaccRates/100)*(infRates/100)*100,bottom=vaccRates,color='m',label='Smittetilfælde (uafhængig)')

R0_wuhan = 2.7
R0_alpha = R0_wuhan*1.5
R0_delta = 4.7
# R0_delta_winter = 4.7*1.2
R0_delta_winter = R0_wuhan * 1.5 * 1.5

herd_wuhan = 1 - (1/R0_wuhan)
herd_alpha = 1 - (1/R0_alpha)
herd_delta = 1 - (1/R0_delta)
herd_delta_winter = 1 - (1/R0_delta_winter)



cmap = plt.cm.get_cmap('cividis',4)
# ax1.plot([-1,len(curCountries)],[100*herd_wuhan,100*herd_wuhan],color=cmap(0),label=f'Flokimmunitet, R0={R0_wuhan:1.1f}\nOriginal virus')
# ax1.plot([-1,len(curCountries)],[100*herd_alpha,100*herd_alpha],color=cmap(1),label=f'Flokimmunitet, R0={R0_alpha:1.1f}\nAlpha-variant')
# ax1.plot([-1,len(curCountries)],[100*herd_delta,100*herd_delta],color=cmap(2),label=f'Flokimmunitet, R0={R0_delta:1.1f}\nDelta-variant (sommer)')
# ax1.plot([-1,len(curCountries)],[100*herd_delta_winter,100*herd_delta_winter],color=cmap(3),label=f'Flokimmunitet, R0={R0_delta_winter:1.1f}\nDelta-variant (vinter)')
ax1.plot([-1,len(curCountries)],[100*herd_wuhan,100*herd_wuhan],color=cmap(0),label=f'Herd immunity, R0={R0_wuhan:1.1f}\nOriginal virus')
ax1.plot([-1,len(curCountries)],[100*herd_alpha,100*herd_alpha],color=cmap(1),label=f'Herd immunity, R0={R0_alpha:1.1f}\nAlpha-variant')
ax1.plot([-1,len(curCountries)],[100*herd_delta,100*herd_delta],color=cmap(2),label=f'Herd immunity, R0={R0_delta:1.1f}\nDelta-variant, summer')
ax1.plot([-1,len(curCountries)],[100*herd_delta_winter,100*herd_delta_winter],color=cmap(3),label=f'Herd immunity, R0={R0_delta_winter:1.1f}\nDelta-variant, winter')

ax1.legend(bbox_to_anchor=(1.04,0.5), loc="center left")
ax1.set_ylim([0,100])
ax1.set_ylim([0,100])
ax1.set_xlim(-0.5,len(curCountries)-0.5)
ax1.set_ylabel('Total rate of immunity in society [%]')
# ax1.set_ylabel('Grad af immunitet i samfundet')

from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
ax1.yaxis.set_minor_locator(MultipleLocator(5)) 
ax1.set_axisbelow(True)
ax1.grid(axis='y',which='both',zorder=0)

plt.xticks(rotation=90)
plt.tight_layout()

plt.savefig(path_figs + 'ImmunityComparison')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:


fig,ax1 = plt.subplots(figsize=(20,8))

fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')


# Sort by vaccinationrate
newOrder = np.argsort(vaccRates)
vaccRatesSort = vaccRates[newOrder]
infRatesSort = infRates[newOrder] 
curCountriesSort = curCountries[newOrder]
immuRateDependentSort = vaccRatesSort + infRatesSort
immuRateIndependentSort = vaccRatesSort + infRatesSort - (vaccRatesSort/100)*(infRatesSort/100)*100
infRateIndependentSort = infRatesSort - (vaccRatesSort/100)*(infRatesSort/100)*100


cmap = plt.cm.get_cmap('magma',5)
ax1.bar(curCountriesSort,vaccRatesSort,color=cmap(0),label='Vaccinations')
ax1.bar(curCountriesSort,infRatesSort,bottom=vaccRatesSort,color=cmap(2),label='Cases (Dependent)')
ax1.bar(curCountriesSort,infRateIndependentSort,bottom=vaccRatesSort,color=cmap(1),label='Cases (Independent)')
# ax1.bar(curCountriesSort,infRatesSort,bottom=vaccRatesSort,color=cmap(1),label='Cases (Dependent)')
# ax1.bar(curCountriesSort,infRatesSort - (vaccRatesSort/100)*(infRatesSort/100)*100,bottom=vaccRatesSort,color=cmap(2),label='Cases (Independent)')
# ax1.bar(curCountriesSort,vaccRatesSort,color=cmap(0),label='Vaccinationer')
# ax1.bar(curCountriesSort,infRatesSort,bottom=vaccRatesSort,color=cmap(1),label='Smittetilfælde (afhængig)')
# ax1.bar(curCountriesSort,infRatesSort - (vaccRatesSort/100)*(infRatesSort/100)*100,bottom=vaccRatesSort,color=cmap(2),label='Smittetilfælde (uafhængig)')
# ax1.bar(curCountriesSort,vaccRatesSort,color='b',label='Vaccinationer')
# ax1.bar(curCountriesSort,infRatesSort,bottom=vaccRatesSort,color='r',label='Smittetilfælde (afhængig)')
# ax1.bar(curCountriesSort,infRatesSort - (vaccRatesSort/100)*(infRatesSort/100)*100,bottom=vaccRatesSort,color='m',label='Smittetilfælde (uafhængig)')

R0_wuhan = 2.7
R0_alpha = R0_wuhan*1.5
R0_delta = 4.7
# R0_delta_winter = 4.7*1.2
R0_delta_winter = R0_wuhan * 1.5 * 1.5

herd_wuhan = 1 - (1/R0_wuhan)
herd_alpha = 1 - (1/R0_alpha)
herd_delta = 1 - (1/R0_delta)
herd_delta_winter = 1 - (1/R0_delta_winter)



cmap = plt.cm.get_cmap('cividis',4)
# ax1.plot([-1,len(curCountriesSort)],[100*herd_wuhan,100*herd_wuhan],color=cmap(0),label=f'Flokimmunitet, R0={R0_wuhan:1.1f}\nOriginal virus')
# ax1.plot([-1,len(curCountriesSort)],[100*herd_alpha,100*herd_alpha],color=cmap(1),label=f'Flokimmunitet, R0={R0_alpha:1.1f}\nAlpha-variant')
# ax1.plot([-1,len(curCountriesSort)],[100*herd_delta,100*herd_delta],color=cmap(2),label=f'Flokimmunitet, R0={R0_delta:1.1f}\nDelta-variant (sommer)')
# ax1.plot([-1,len(curCountriesSort)],[100*herd_delta_winter,100*herd_delta_winter],color=cmap(3),label=f'Flokimmunitet, R0={R0_delta_winter:1.1f}\nDelta-variant (vinter)')
ax1.plot([-1,len(curCountriesSort)],[100*herd_wuhan,100*herd_wuhan],color=cmap(0),label=f'Herd immunity, R0={R0_wuhan:1.1f}\nOriginal virus')
ax1.plot([-1,len(curCountriesSort)],[100*herd_alpha,100*herd_alpha],color=cmap(1),label=f'Herd immunity, R0={R0_alpha:1.1f}\nAlpha-variant')
ax1.plot([-1,len(curCountriesSort)],[100*herd_delta,100*herd_delta],color=cmap(2),label=f'Herd immunity, R0={R0_delta:1.1f}\nDelta-variant, summer')
ax1.plot([-1,len(curCountriesSort)],[100*herd_delta_winter,100*herd_delta_winter],color=cmap(3),label=f'Herd immunity, R0={R0_delta_winter:1.1f}\nDelta-variant, winter')

ax1.legend(bbox_to_anchor=(1.04,0.5), loc="center left")
ax1.set_ylim([0,100])
ax1.set_ylim([0,100])
ax1.set_xlim(-0.5,len(curCountries)-0.5)
ax1.set_ylabel('Total rate of immunity in society [%]')
# ax1.set_ylabel('Grad af immunitet i samfundet')

from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
ax1.yaxis.set_minor_locator(MultipleLocator(5)) 
ax1.set_axisbelow(True)
ax1.grid(axis='y',which='both',zorder=0)

plt.xticks(rotation=90)
plt.tight_layout()

plt.savefig(path_figs + 'ImmunityComparisonSortVacc')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
df[df.location == 'Moldova'].iloc[-10:,35:55]

,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence
79368,NaN,887228.0,NaN,4407.0,4253.0,38.35,NaN,22.05,NaN,1057.0,NaN,4024025.0,123.655,37.6,10.864,6.955,5189.972,0.2,408.502,5.72
79369,NaN,888318.0,NaN,1832.0,3835.0,38.40,NaN,22.08,NaN,953.0,NaN,4024025.0,123.655,37.6,10.864,6.955,5189.972,0.2,408.502,5.72
79370,NaN,888853.0,NaN,931.0,3641.0,38.42,NaN,22.09,NaN,905.0,NaN,4024025.0,123.655,37.6,10.864,6.955,5189.972,0.2,408.502,5.72
79371,NaN,890666.0,NaN,2881.0,3300.0,38.49,NaN,22.13,NaN,820.0,NaN,4024025.0,123.655,37.6,10.864,6.955,5189.972,0.2,408.502,5.72
79372,NaN,NaN,NaN,NaN,3236.0,NaN,NaN,NaN,NaN,804.0,NaN,4024025.0,123.655,37.6,10.864,6.955,5189.972,0.2,408.502,5.72
79373,NaN,895607.0,NaN,NaN,3200.0,38.69,NaN,22.26,NaN,795.0,NaN,4024025.0,123.655,37.6,10.864,6.955,5189.972,0.2,408.502,5.72
79374,NaN,NaN,NaN,NaN,3061.0,NaN,NaN,NaN,NaN,761.0,NaN,4024025.0,123.655,37.6,10.864,6.955,5189.972,0.2,408.502,5.72
79375,NaN,NaN,NaN,NaN,2909.0,NaN,NaN,NaN,NaN,723.0,NaN,4024025.0,123.655,37.6,10.864,6.955,5189.972,0.2,408.502,5.72
79376,NaN,901624.0,NaN,NaN,3124.0,38.94,NaN,22.41,NaN,776.0,NaN,4024025.0,123.655,37.6,10.864,6.955,5189.972,0.2,408.502,5.72
79377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4024025.0,123.655,37.6,10.864,6.955,5189.972,0.2,408.502,5.72


In [15]:
fig,ax1 = plt.subplots(figsize=(15,23))

fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')


cmap = plt.cm.get_cmap('magma',5)
ax1.barh(curCountries[::-1],vaccRates[::-1],height=0.75,edgecolor='k',color=cmap(0),label='Vaccinations')
ax1.barh(curCountries[::-1],infRates[::-1],left=vaccRates[::-1],height=0.75,edgecolor='k',color=cmap(2),label='Cases (Dependent)')
ax1.barh(curCountries[::-1],infRateIndependent[::-1],left=vaccRates[::-1],height=0.75,edgecolor='k',color=cmap(1),label='Cases (Independent)')

R0_wuhan = 2.7
R0_alpha = R0_wuhan*1.5
R0_delta = 4.7
# R0_delta_winter = 4.7*1.2
R0_delta_winter = R0_wuhan * 1.5 * 1.5
R0_delta_winter2 = R0_wuhan * 1.5 * 1.6

herd_wuhan = 1 - (1/R0_wuhan)
herd_alpha = 1 - (1/R0_alpha)
herd_delta = 1 - (1/R0_delta)
herd_delta_winter = 1 - (1/R0_delta_winter)
herd_delta_winter2 = 1 - (1/R0_delta_winter2)

# ax1.fill([0,20],[0,80],'k')
numCoun = len(curCountries)
cmap = plt.cm.get_cmap('cividis',6)
ax1.fill_between([0,100*herd_wuhan],[numCoun+1,numCoun+1],[-1,-1],color=cmap(2),zorder=-1,hatch='///',edgecolor='k',label=f'Herd immunity, R0={R0_wuhan:1.1f}\nOriginal virus') 
ax1.fill_between([0,100*herd_alpha],[numCoun+1,numCoun+1],[-1,-1],color=cmap(3),zorder=-2,hatch='\\',edgecolor='k',label=f'Herd immunity, R0={R0_alpha:1.1f}\nAlpha-variant')
ax1.fill_between([0,100*herd_delta],[numCoun+1,numCoun+1],[-1,-1],color=cmap(4),zorder=-3,hatch='x',edgecolor='k',label=f'Herd immunity, R0={R0_delta:1.1f}\nDelta-variant, summer')
ax1.fill_between([0,100*herd_delta_winter],[numCoun+1,numCoun+1],[-1,-1],color=[1,0.2,0.2],zorder=-4,hatch='/',edgecolor='k',label=f'Herd immunity, R0={R0_delta_winter:1.1f}\nDelta-variant, winter')
# ax1.fill_between([0,100*herd_delta_winter2],[numCoun+1,numCoun+1],[-1,-1],color=[1,0.2,0.2,0.5],zorder=-5,hatch='/')
# ax1.fill_between([0,100*herd_delta_winter2 + 5],[numCoun+1,numCoun+1],[-1,-1],color=[1,0.2,0.2,0.25],zorder=-6,hatch='/')

ax1.set_ylim([-1,numCoun])

# cmap = plt.cm.get_cmap('cividis',4)
# # ax1.plot([-1,len(curCountries)],[100*herd_wuhan,100*herd_wuhan],color=cmap(0),label=f'Flokimmunitet, R0={R0_wuhan:1.1f}\nOriginal virus') 
# # ax1.plot([-1,len(curCountries)],[100*herd_alpha,100*herd_alpha],color=cmap(1),label=f'Flokimmunitet, R0={R0_alpha:1.1f}\nAlpha-variant')
# # ax1.plot([-1,len(curCountries)],[100*herd_delta,100*herd_delta],color=cmap(2),label=f'Flokimmunitet, R0={R0_delta:1.1f}\nDelta-variant (sommer)')
# # ax1.plot([-1,len(curCountries)],[100*herd_delta_winter,100*herd_delta_winter],color=cmap(3),label=f'Flokimmunitet, R0={R0_delta_winter:1.1f}\nDelta-variant (vinter)')
# ax1.plot([-1,len(curCountries)],[100*herd_wuhan,100*herd_wuhan],color=cmap(0),label=f'Herd immunity, R0={R0_wuhan:1.1f}\nOriginal virus')
# ax1.plot([-1,len(curCountries)],[100*herd_alpha,100*herd_alpha],color=cmap(1),label=f'Herd immunity, R0={R0_alpha:1.1f}\nAlpha-variant')
# ax1.plot([-1,len(curCountries)],[100*herd_delta,100*herd_delta],color=cmap(2),label=f'Herd immunity, R0={R0_delta:1.1f}\nDelta-variant (summer)')
# ax1.plot([-1,len(curCountries)],[100*herd_delta_winter,100*herd_delta_winter],color=cmap(3),label=f'Herd immunity, R0={R0_delta_winter:1.1f}\nDelta-variant (winter)')

ax1.legend(bbox_to_anchor=(1.04,0.5), loc="center left")
# ax1.set_ylim([0,100])
ax1.set_xlim([0,100])
# ax1.set_xlim(-0.5,len(curCountries)-0.5)
ax1.set_xlabel('Total rate of immunity in society [%]')
# # ax1.set_ylabel('Grad af immunitet i samfundet')

# from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
# ax1.yaxis.set_minor_locator(MultipleLocator(5)) 
# ax1.set_axisbelow(True)
# ax1.grid(axis='y',which='both',zorder=0)

# plt.xticks(rotation=90)
plt.tight_layout()

plt.savefig(path_figs + 'ImmunityComparison2')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
fig,ax1 = plt.subplots(figsize=(15,23))

fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')


cmap = plt.cm.get_cmap('magma',5)
ax1.barh(curCountriesSort[::-1],vaccRatesSort[::-1],height=0.75,edgecolor='k',color=cmap(0),label='Vaccinations')
ax1.barh(curCountriesSort[::-1],infRatesSort[::-1],left=vaccRatesSort[::-1],height=0.75,edgecolor='k',color=cmap(2),label='Cases (Dependent)')
ax1.barh(curCountriesSort[::-1],infRateIndependentSort[::-1],left=vaccRatesSort[::-1],height=0.75,edgecolor='k',color=cmap(1),label='Cases (Independent)')

R0_wuhan = 2.7
R0_alpha = R0_wuhan*1.5
R0_delta = 4.7
# R0_delta_winter = 4.7*1.2
R0_delta_winter = R0_wuhan * 1.5 * 1.5
R0_delta_winter2 = R0_wuhan * 1.5 * 1.6

herd_wuhan = 1 - (1/R0_wuhan)
herd_alpha = 1 - (1/R0_alpha)
herd_delta = 1 - (1/R0_delta)
herd_delta_winter = 1 - (1/R0_delta_winter)
herd_delta_winter2 = 1 - (1/R0_delta_winter2)

# ax1.fill([0,20],[0,80],'k')
numCoun = len(curCountriesSort)
cmap = plt.cm.get_cmap('cividis',6)
ax1.fill_between([0,100*herd_wuhan],[numCoun+1,numCoun+1],[-1,-1],color=cmap(2),zorder=-1,hatch='///',edgecolor='k',label=f'Herd immunity, R0={R0_wuhan:1.1f}\nOriginal virus') 
ax1.fill_between([0,100*herd_alpha],[numCoun+1,numCoun+1],[-1,-1],color=cmap(3),zorder=-2,hatch='\\',edgecolor='k',label=f'Herd immunity, R0={R0_alpha:1.1f}\nAlpha-variant')
ax1.fill_between([0,100*herd_delta],[numCoun+1,numCoun+1],[-1,-1],color=cmap(4),zorder=-3,hatch='x',edgecolor='k',label=f'Herd immunity, R0={R0_delta:1.1f}\nDelta-variant, summer')
ax1.fill_between([0,100*herd_delta_winter],[numCoun+1,numCoun+1],[-1,-1],color=[1,0.2,0.2],zorder=-4,hatch='/',edgecolor='k',label=f'Herd immunity, R0={R0_delta_winter:1.1f}\nDelta-variant, winter')

ax1.set_ylim([-1,numCoun])

ax1.legend(bbox_to_anchor=(1.04,0.5), loc="center left")
# ax1.set_ylim([0,100])
ax1.set_xlim([0,100])
# ax1.set_xlim(-0.5,len(curCountriesSort)-0.5)
ax1.set_xlabel('Total rate of immunity in society [%]')
# # ax1.set_ylabel('Grad af immunitet i samfundet')

# plt.xticks(rotation=90)
plt.tight_layout()

plt.savefig(path_figs + 'ImmunityComparison2Sorted')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
# df[df.location == 'Sweden'].columns

print(df[df.location == 'Denmark']['people_fully_vaccinated_per_hundred'].tail(10))
print(df[df.location == 'Denmark']['total_cases_per_million'].tail(10)/1000000)
print(df[df.location == 'Sweden']['people_fully_vaccinated_per_hundred'].tail(10))
print(df[df.location == 'Sweden']['total_cases_per_million'].tail(10)/1000000)

32388    76.00
32389    76.02
32390    76.02
32391    76.04
32392    76.07
32393    76.11
32394    76.14
32395      NaN
32396      NaN
32397      NaN
Name: people_fully_vaccinated_per_hundred, dtype: float64
32388    0.069056
32389    0.069425
32390    0.069846
32391    0.070253
32392    0.070712
32393    0.071246
32394    0.071616
32395    0.072406
32396    0.072935
32397    0.073495
Name: total_cases_per_million, dtype: float64
115434    68.23
115435      NaN
115436      NaN
115437      NaN
115438    68.38
115439    68.42
115440    68.49
115441    68.55
115442      NaN
115443      NaN
Name: people_fully_vaccinated_per_hundred, dtype: float64
115434    0.115854
115435    0.115854
115436    0.115854
115437    0.115854
115438    0.116060
115439    0.116191
115440    0.116291
115441    0.116383
115442    0.116383
115443    0.116383
Name: total_cases_per_million, dtype: float64


# Single country over time
Idea: Show total immunity for a country, compared with the herd immunity threshold at the time.


In [18]:
curCountry = 'Denmark'
# curCountry = 'Lithuania'
# curCountry = 'Germany'
curdf = df[df.location == curCountry]

curdf.columns
curVacc = curdf.people_fully_vaccinated_per_hundred/100
curInfi = curdf.total_cases_per_million/1000000

# Drop whenever there is a nan value and reload variables
# curdf = curdf[(~curVacc.isna()) & (~curInfi.isna())]
# curdf = curdf[(~curVacc.isna())]
# curVacc = curdf.people_fully_vaccinated_per_hundred/100
# curInfi = curdf.total_cases_per_million/1000000

curDate = pd.to_datetime(curdf.date)

curVacc[curVacc.isna()] = 0

curImmuDep = curVacc + curInfi - curVacc * curInfi
curImmuInd = curVacc + curInfi 

# Variant thresholds
R0_wuhan = 2.7
R0_alpha = R0_wuhan*1.5
R0_delta = 4.7
R0_delta_winter = R0_wuhan * 1.5 * 1.5
R0_delta_winter2 = R0_wuhan * 1.5 * 1.6

herd_wuhan = 1 - (1/R0_wuhan)
herd_alpha = 1 - (1/R0_alpha)
herd_delta = 1 - (1/R0_delta)
herd_delta_winter = 1 - (1/R0_delta_winter)
herd_delta_winter2 = 1 - (1/R0_delta_winter2)

# Make figure
fig,ax1 = plt.subplots()
fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')

label_inf = 'Immunitet gennem smitte'
label_vac = 'Immunitet gennem vaccine'
label_vacInd = 'Immunitet gennem vaccine og smitte (nedre grænse)'
label_vacDep = 'Immunitet gennem vaccine og smitte (øvre grænse)'
label_y = 'Samlet immunitet [%]'

# # Infections at the bottom
# ax1.fill_between(curDate,100*curInfi,color='xkcd:Red',label=label_inf)
# ax1.fill_between(curDate,100*curImmuInd,100*curInfi,color='xkcd:Blue',label=label_vacInd)
# ax1.fill_between(curDate,100*curImmuDep,100*curInfi,color='xkcd:DarkBlue',label=label_vacDep)

# Vaccinations at the bottom
ax1.fill_between(curDate,100*curVacc,color='xkcd:DarkBlue',label=label_vac)
ax1.fill_between(curDate,100*curImmuInd,100*curVacc,color='xkcd:Yellow',label=label_vacInd)
ax1.fill_between(curDate,100*curImmuDep,100*curVacc,color='xkcd:Red',label=label_vacDep)
# ax1.fill_between(curDate,100*(curVacc + curInfi),100*curVacc,color='xkcd:Red',label='Smittede (uvaccinerede)')
# ax1.fill_between(curDate,100*(curVacc + curVacc * curInfi),100*curVacc,color='xkcd:Yellow',label='Smittede (vaccinerede)')

# Plot immunity threshold
alpha_date = np.datetime64('2021-02-15')
delta_date = np.datetime64('2021-06-20')
delta_winter_date = np.datetime64('2021-10-01')

firstDate = curDate.values[0]
lastDate = curDate.values[-1]
# Wuhan-strain
ax1.plot([firstDate,alpha_date],[herd_wuhan*100,herd_wuhan*100],'k',label='Original virus')
ax1.plot([alpha_date,lastDate],[herd_wuhan*100,herd_wuhan*100],'k:')
ax1.plot([alpha_date,alpha_date],[herd_wuhan*100,herd_alpha*100],'k')
# Alpha
ax1.plot([alpha_date,delta_date],[herd_alpha*100,herd_alpha*100],color='xkcd:maroon',label='Alpha-variant')
ax1.plot([delta_date,lastDate],[herd_alpha*100,herd_alpha*100],':',color='xkcd:maroon')
ax1.plot([delta_date,delta_date],[herd_alpha*100,herd_delta*100],color='xkcd:maroon')
# Delta
ax1.plot([delta_date,delta_winter_date],[herd_delta*100,herd_delta*100],'r',label='Delta-variant')
ax1.plot([delta_winter_date,lastDate],[herd_delta*100,herd_delta*100],'r:')
ax1.plot([delta_winter_date,delta_winter_date],[herd_delta*100,herd_delta_winter*100],'r')
# Delta, winter
ax1.plot([delta_winter_date,lastDate],[herd_delta_winter*100,herd_delta_winter*100],'m',label='Delta-variant (winter)')


# Make figure nice
ax1.legend(loc='center left',bbox_to_anchor=(0,0.4))
# ax1.grid(axis='y')

from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
ax1.yaxis.set_major_locator(MultipleLocator(10)) 
ax1.yaxis.set_minor_locator(MultipleLocator(5)) 
ax1.set_axisbelow(True)
ax1.grid(axis='y',which='both',zorder=0)

ax1.set_ylabel(label_y)
ax1.set_xlim(left=firstDate,right=curDate.values[-5])
ax1.set_ylim(bottom=0,top=100)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
# fig,ax1 = plt.subplots()
# curDates = pd.to_datetime(curdf.date).values
# ax1.plot(curDates,curdf.new_cases,'k.-',markersize=2,linewidth=0.5)
# ax1.plot(rnTime(curDates,7),rnMean(curdf.new_cases,7),'k')
# ax1.set_ylim(bottom=0)
# fig.tight_layout()


In [20]:
# Make figure
fig,(ax1,ax2) = plt.subplots(2,1,sharex=True)
fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')
ax2.set_facecolor('lightgrey')

# Vaccinations at the bottom
ax1.fill_between(curDate,100*curVacc,color='xkcd:DarkBlue',label=label_vac)
ax1.fill_between(curDate,100*curImmuInd,100*curVacc,color='xkcd:Yellow',label=label_vacInd)
ax1.fill_between(curDate,100*curImmuDep,100*curVacc,color='xkcd:Red',label=label_vacDep)
# ax1.fill_between(curDate,100*(curVacc + curInfi),100*curVacc,color='xkcd:Red',label='Smittede (uvaccinerede)')
# ax1.fill_between(curDate,100*(curVacc + curVacc * curInfi),100*curVacc,color='xkcd:Yellow',label='Smittede (vaccinerede)')

firstDate = curDate.values[0]
lastDate = curDate.values[-1]
# Wuhan-strain
ax1.plot([firstDate,alpha_date],[herd_wuhan*100,herd_wuhan*100],'k',label='Original virus')
ax1.plot([alpha_date,lastDate],[herd_wuhan*100,herd_wuhan*100],'k:')
ax1.plot([alpha_date,alpha_date],[herd_wuhan*100,herd_alpha*100],'k')
# Alpha
ax1.plot([alpha_date,delta_date],[herd_alpha*100,herd_alpha*100],color='xkcd:maroon',label='Alpha-variant')
ax1.plot([delta_date,lastDate],[herd_alpha*100,herd_alpha*100],':',color='xkcd:maroon')
ax1.plot([delta_date,delta_date],[herd_alpha*100,herd_delta*100],color='xkcd:maroon')
# Delta
ax1.plot([delta_date,delta_winter_date],[herd_delta*100,herd_delta*100],'r',label='Delta-variant')
ax1.plot([delta_winter_date,lastDate],[herd_delta*100,herd_delta*100],'r:')
ax1.plot([delta_winter_date,delta_winter_date],[herd_delta*100,herd_delta_winter*100],'r')
# Delta, winter
ax1.plot([delta_winter_date,lastDate],[herd_delta_winter*100,herd_delta_winter*100],'m',label='Delta-variant (vinter)')

curDatesAlle = pd.to_datetime(curdf.date).values
ax2.plot(curDatesAlle,curdf.new_cases,'k.-',markersize=2,linewidth=0.5)
ax2.plot(rnTime(curDatesAlle,7),rnMean(curdf.new_cases,7),'k')
ax2.set_ylim(bottom=0)

# Make figure nice
ax1.legend(loc='center left',bbox_to_anchor=(0,0.3),fontsize=10)
ax2.grid(axis='y')

from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
ax1.yaxis.set_major_locator(MultipleLocator(20)) 
ax1.yaxis.set_minor_locator(MultipleLocator(10)) 
ax1.set_axisbelow(True)
ax1.grid(axis='y',which='both',zorder=0)

ax1.set_ylabel(label_y)
ax1.set_xlim(left=firstDate,right=curDate.values[-4])
ax1.set_ylim(bottom=0,top=100)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Testing different ways split up "old" immunity


In [60]:
df[~df.total_boosters_per_hundred.isna()].location.unique()

array(['Africa', 'Albania', 'Asia', 'Australia', 'Austria', 'Azerbaijan',
       'Belgium', 'Bermuda', 'Bolivia', 'Brazil', 'Bulgaria', 'Cambodia',
       'Canada', 'Chile', 'China', 'Costa Rica', 'Cyprus', 'Czechia',
       'Dominican Republic', 'Ecuador', 'El Salvador', 'Europe',
       'European Union', 'France', 'Germany', 'Gibraltar', 'Greece',
       'High income', 'Honduras', 'Hungary', 'Iceland', 'Iran', 'Israel',
       'Italy', 'Lithuania', 'Lower middle income', 'Luxembourg',
       'Malaysia', 'Malta', 'North America', 'North Macedonia',
       'Northern Cyprus', 'Oceania', 'Palestine', 'Paraguay', 'Peru',
       'Poland', 'Russia', 'Serbia', 'Singapore', 'South America',
       'South Korea', 'Spain', 'Thailand', 'Tunisia', 'Turkey',
       'United Arab Emirates', 'United Kingdom', 'United States',
       'Upper middle income', 'Uruguay', 'World'], dtype=object)

In [61]:
fig,ax1 = plt.subplots()


curdf = df[df.location == 'Germany']
thisdates = pd.to_datetime(curdf.date)
ax1.fill_between(thisdates,curdf.people_fully_vaccinated_per_hundred,color='k')
ax1.fill_between(thisdates.values+np.timedelta64(int(366/2),'D'),curdf.people_fully_vaccinated_per_hundred,color='xkcd:Grey')
ax1.fill_between(thisdates,curdf.total_boosters_per_hundred)
ax1.set_xlim(right=thisdates.max()-np.timedelta64(3,'D'))

# curdf.columns
# thisdates.values+np.timedelta64(10,'D')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(18597.85, 18942.0)

datetime.date(2021, 11, 19)

In [ ]:
# curCountry = curCountries[0]

vaccRates = []
infRates = []
# for curCountry in curCountries:
curCountries = []
for curCountry in allCountries:
    curdf = df[df.location == curCountry]

    curInf = curdf.total_cases_per_million
    curVacc = curdf.people_vaccinated_per_hundred
    if len(curInf[~curInf.isna()]) == 0:
        continue
    if len(curVacc[~curVacc.isna()]) == 0:
        continue
    
    latestInfRate = curInf[~curInf.isna()].iloc[-1]
    infRates.append(100*latestInfRate/1000000) # Per million, changed to per hundred
    
    # curLastRow = curdf[curdf.date == (curdf.date.max())]
    # curdf.people_vaccinated_per_hundred
    latestVaccRate = curVacc[~curVacc.isna()].iloc[-1]
    vaccRates.append(latestVaccRate)
    
    # If loop didn't break yet, add to list of countries to show
    curCountries.append(curCountry)
    
    

vaccRates = np.array(vaccRates)    
infRates = np.array(infRates)
curCountries = np.array(curCountries)

# # Sort by vaccinationrate
# newOrder = np.argsort(vaccRates)
# vaccRates = vaccRates[newOrder]
# infRates = infRates[newOrder] 
# curCountries = curCountries[newOrder]

immuRateDependent = vaccRates + infRates
immuRateIndependent = vaccRates + infRates - (vaccRates/100)*(infRates/100)*100
infRateIndependent = infRates - (vaccRates/100)*(infRates/100)*100

In [ ]:
# Testing different ways to show it


fig,ax1 = plt.subplots(figsize=(20,8))

fig.patch.set_facecolor('lightgrey')
ax1.set_facecolor('lightgrey')

cmap = plt.cm.get_cmap('magma',5)
ax1.bar(curCountries,vaccRates,color=cmap(0),label='Vaccinations')
ax1.bar(curCountries,infRates,bottom=vaccRates,color=cmap(2),label='Cases (Dependent)')
ax1.bar(curCountries,infRateIndependent,bottom=vaccRates,color=cmap(1),label='Cases (Independent)')
# ax1.bar(curCountries,infRates,bottom=vaccRates,color=cmap(1),label='Cases (Dependent)')
# ax1.bar(curCountries,infRates - (vaccRates/100)*(infRates/100)*100,bottom=vaccRates,color=cmap(2),label='Cases (Independent)')
# ax1.bar(curCountries,vaccRates,color=cmap(0),label='Vaccinationer')
# ax1.bar(curCountries,infRates,bottom=vaccRates,color=cmap(1),label='Smittetilfælde (afhængig)')
# ax1.bar(curCountries,infRates - (vaccRates/100)*(infRates/100)*100,bottom=vaccRates,color=cmap(2),label='Smittetilfælde (uafhængig)')
# ax1.bar(curCountries,vaccRates,color='b',label='Vaccinationer')
# ax1.bar(curCountries,infRates,bottom=vaccRates,color='r',label='Smittetilfælde (afhængig)')
# ax1.bar(curCountries,infRates - (vaccRates/100)*(infRates/100)*100,bottom=vaccRates,color='m',label='Smittetilfælde (uafhængig)')

R0_wuhan = 2.7
R0_alpha = R0_wuhan*1.5
R0_delta = 4.7
# R0_delta_winter = 4.7*1.2
R0_delta_winter = R0_wuhan * 1.5 * 1.5

herd_wuhan = 1 - (1/R0_wuhan)
herd_alpha = 1 - (1/R0_alpha)
herd_delta = 1 - (1/R0_delta)
herd_delta_winter = 1 - (1/R0_delta_winter)



cmap = plt.cm.get_cmap('cividis',4)
# ax1.plot([-1,len(curCountries)],[100*herd_wuhan,100*herd_wuhan],color=cmap(0),label=f'Flokimmunitet, R0={R0_wuhan:1.1f}\nOriginal virus')
# ax1.plot([-1,len(curCountries)],[100*herd_alpha,100*herd_alpha],color=cmap(1),label=f'Flokimmunitet, R0={R0_alpha:1.1f}\nAlpha-variant')
# ax1.plot([-1,len(curCountries)],[100*herd_delta,100*herd_delta],color=cmap(2),label=f'Flokimmunitet, R0={R0_delta:1.1f}\nDelta-variant (sommer)')
# ax1.plot([-1,len(curCountries)],[100*herd_delta_winter,100*herd_delta_winter],color=cmap(3),label=f'Flokimmunitet, R0={R0_delta_winter:1.1f}\nDelta-variant (vinter)')
ax1.plot([-1,len(curCountries)],[100*herd_wuhan,100*herd_wuhan],color=cmap(0),label=f'Herd immunity, R0={R0_wuhan:1.1f}\nOriginal virus')
ax1.plot([-1,len(curCountries)],[100*herd_alpha,100*herd_alpha],color=cmap(1),label=f'Herd immunity, R0={R0_alpha:1.1f}\nAlpha-variant')
ax1.plot([-1,len(curCountries)],[100*herd_delta,100*herd_delta],color=cmap(2),label=f'Herd immunity, R0={R0_delta:1.1f}\nDelta-variant, summer')
ax1.plot([-1,len(curCountries)],[100*herd_delta_winter,100*herd_delta_winter],color=cmap(3),label=f'Herd immunity, R0={R0_delta_winter:1.1f}\nDelta-variant, winter')

ax1.legend(bbox_to_anchor=(1.04,0.5), loc="center left")
ax1.set_ylim([0,100])
ax1.set_ylim([0,100])
ax1.set_xlim(-0.5,len(curCountries)-0.5)
ax1.set_ylabel('Total rate of immunity in society [%]')
# ax1.set_ylabel('Grad af immunitet i samfundet')

from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
ax1.yaxis.set_minor_locator(MultipleLocator(5)) 
ax1.set_axisbelow(True)
ax1.grid(axis='y',which='both',zorder=0)

plt.xticks(rotation=90)
plt.tight_layout()

# plt.savefig(path_figs + 'ImmunityComparison')